In [35]:
import numpy as np
import pandas as pd
from typing import List
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
from collections import defaultdict
from sklearn.cluster import KMeans
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz

c:\Users\milan\OneDrive\Documents\portfolio\entity_resolution\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [45]:
class Simmat:
    def __init__(self, sim_object):
        self.mat = np.array([], dtype=float)
        self.nodes = []
        self.sim_object = sim_object

    def add_node(self, node: str, compare_to_ids: List[int] = None):
        """
        compare_to_ids: only compare <node> to respective other nodes,
        useful for blocking
        """
        assert node not in self.nodes, "node already exists"
        if compare_to_ids is None:
            compare_to_ids = range(len(self.nodes))
        self.nodes.append(node)
        node_sims = np.zeros(len(self.nodes), dtype=float)
        for i in compare_to_ids:
            node_other = self.nodes[i]
            node_sims[i] = self.sim_object.sim(node, node_other)
        if tuple(self.mat.shape) == (0,):
            self.mat = np.array([1], dtype=float).reshape(1, 1)
        else:
            shape = self.mat.shape[0]
            mat_new = np.zeros((shape + 1, shape + 1))
            mat_new[:shape, :shape] = self.mat
            mat_new[-1, :] = node_sims
            mat_new[:, -1] = node_sims
            self.mat = mat_new

    def remove_node(self, node: str):
        node_id = self.nodes.index(node)
        self.nodes.remove(node)
        self.mat = np.delete(self.mat, node_id, axis=0)
        self.mat = np.delete(self.mat, node_id, axis=1)


class CharNgramBlock:
    def __init__(
            self, 
            n: int = 3, 
            chars: str = 'abcdefghijklmnopqrstuvwxyzäöüß.-_1234567890 '):
        self.vocabulary = [''.join(triple) for triple in itertools.product(chars, repeat=n)]
        self.vectorizer = TfidfVectorizer(
            #vocabulary=self.vocabulary, 
            analyzer=lambda text: self.char_ngrams(text, n))

    @staticmethod
    def char_ngrams(node: str, n: int) -> list[str]:
        return [node[i:i+n] for i in range(len(node)-n+1)]

    def block(self, nodes: pd.Series, n_blocks: int) -> List[List[int]]:
        tfidf_matrix = self.vectorizer.fit_transform(nodes)
        preds = KMeans(n_clusters=n_blocks, n_init=10).fit_predict(tfidf_matrix)
        value_to_ids = defaultdict(list)
        for idx, value in enumerate(preds):
            value_to_ids[value].append(idx)
        return dict(value_to_ids), preds


class CharNgramSim:
    def __init__(
            self, 
            n: int = 3, 
            chars: str = 'abcdefghijklmnopqrstuvwxyzäöüß.-_1234567890 ', 
            precision: int = 2):
        self.precision = precision
        self.vocabulary = [''.join(triple) for triple in itertools.product(chars, repeat=n)]
        self.vectorizer = TfidfVectorizer(
            vocabulary=self.vocabulary, 
            binary=True, 
            use_idf=False, 
            norm=None, 
            analyzer=lambda text: self.char_ngrams(text, n))
        self.vectorizer.fit(["text"])

    @staticmethod
    def char_ngrams(node: str, n: int) -> list[str]:
        return [node[i:i+n] for i in range(len(node)-n+1)]
    
    def sim(self, node_i: str, node_j: str) -> int:
        vector1, vector2 = self.vectorizer.transform([node_i, node_j])
        return round(1 - (np.abs(vector1.todense() - vector2.todense()).sum() / (vector1.sum() + vector2.sum())), self.precision)


class EditSim:
    def __init__(
            self, 
            precision: int = 2):
        self.precision = precision

    @staticmethod
    def char_ngrams(node: str, n: int) -> list[str]:
        return [node[i:i+n] for i in range(len(node)-n+1)]
    
    def sim(self, node_i: str, node_j: str) -> int:
        return round(1 - difflib.SequenceMatcher(None, node_i, node_j).ratio(), self.precision)
 

class EditPartialSim:
    def __init__(
            self, 
            precision: int = 2):
        self.precision = precision

    @staticmethod
    def char_ngrams(node: str, n: int) -> list[str]:
        return [node[i:i+n] for i in range(len(node)-n+1)]
    
    def sim(self, node_i: str, node_j: str) -> int:
        return round(fuzz.partial_ratio(node_i, node_j), self.precision)
    

class EditSetSim:
    def __init__(
            self, 
            precision: int = 2):
        self.precision = precision

    @staticmethod
    def char_ngrams(node: str, n: int) -> list[str]:
        return [node[i:i+n] for i in range(len(node)-n+1)]
    
    def sim(self, node_i: str, node_j: str) -> int:
        return round(fuzz.token_set_ratio(node_i, node_j), self.precision)
    

class TfidfSim:
    def __init__(
            self,
            nodes: List[str],
            precision: int = 2):
        self.vectors = TfidfVectorizer().fit_transform(nodes) 
        self.precision = precision
        self.nodes = nodes


    def sim(self, node_i: str, node_j: str) -> int:
        i = self.nodes.index(node_i)
        j = self.nodes.index(node_j)
        return round(cosine_similarity(self.vectors[i], self.vectors[j])[0, 0], self.precision)


class SimNet:
    def __init__(self, nodes: List[str], sim_mat, edge_thresh: float, use_cliques: bool = False):
        sim_mat = sim_mat * (np.triu(np.ones((len(sim_mat), len(sim_mat))), k=0)) - np.eye(N=len(sim_mat))
        edge_ids = np.argwhere(sim_mat > edge_thresh)
        g = nx.Graph()
        for e in edge_ids:
            node_i = nodes[e[0]]
            node_j = nodes[e[1]]
            g.add_edge(node_i, node_j, weight=sim_mat[e])
            g.add_edge(node_j, node_i, weight=sim_mat[e])
        for n in [n for n in range(len(nodes)) if n not in np.unique(edge_ids)]:
            g.add_node(nodes[n])
        self.g = g
        if use_cliques:
            self.cliques = list(nx.find_cliques(g))
        components = [component for component in nx.connected_components(g)]
        component_map = pd.DataFrame({"node": nodes})
        component_map["component"] = None
        component_map["len"] = None
        self.biggest_component_size = 0
        for i, component in enumerate(components):
            component_map.loc[component_map["node"].isin(component), "component"] = i
            if len(component) > self.biggest_component_size:
                self.biggest_component_size = len(component)
        self.component_lengths = [len(component) for component in components]
        self.component_map = pd.Series(component_map["component"].values, index=component_map["node"].values)

    def sim_cn_jaccard(self, node_i: str, node_j: str, weighted: bool = False) -> float:
        # individual neighbors
        neighbors_i = set(self.g.neighbors(node_i))
        neighbors_j = set(self.g.neighbors(node_j))
        union = neighbors_i.union(neighbors_j)
        intersection = neighbors_i.intersection(neighbors_j)
        if weighted:
            weights_total_i = sum([self.g.get_edge_data(node_i, n)["weight"] for n in neighbors_i])
            weights_total_j = sum([self.g.get_edge_data(node_j, n)["weight"] for n in neighbors_j])
            weights_total = weights_total_i + weights_total_j
        else:
            weights_total = len(union)
        # common neighbors
        if weighted:
            weights_common = 0
            for neighbor in intersection:
                # Sum of edge weights from node_i and node_j to the common neighbor
                weight_i = self.g[node_i][neighbor]['weight']
                weight_j = self.g[node_j][neighbor]['weight']
                weights_common += weight_i + weight_j
        else:
            weights_common = len(intersection)
        if weights_total == 0:
            return 0
        return weights_common / weights_total
    
    def sim_clique_jaccard(self, node_i: str, node_j: str) -> float:
        cliques_i = set([i for i, c in enumerate(self.cliques) if node_i in c])
        cliques_j = set([i for i, c in enumerate(self.cliques) if node_j in c])
        union = cliques_i.union(cliques_j)
        intersection = cliques_i.intersection(cliques_j)
        if len(union) == 0:
            return 0
        return len(intersection) / len(union)
    
    def sim_preferential_attachment(self, node_i: str, node_j: str) -> float:
        return self.g.degree(node_i) * self.g.degree(node_j)
    
    def sim_component(self, node_i: str, node_j: str) -> float:
        """
        if two nodes are in different components, they are unsimilar
        if two nodes are within the same component, 
        they are likely more similar, the smaller the component
        """
        if self.component_map[node_i] == self.component_map[node_j]: 
            component_len = self.component_lengths[self.component_map[node_i]]
            return 1 - (component_len / (self.biggest_component_size + 1))
        return 0
    
    def shortest_path(self, node_i: str, node_j: str) -> int:
        try:
            shortest_path = nx.shortest_path(self.g, source=node_i, target=node_j)
            return len(shortest_path)
        except nx.NetworkXNoPath:
            return None

In [20]:
# preprocess data
n = 1_000


import re

df = pd.read_csv("data/music/musicbrainz-20000-A01.csv")
view = df[df["language"].isin(["German", "Ger.", "german", "geerman", "Geerman", "GERMAN"])][["CID", "title"]]
view.dropna(inplace=True)
view["title"] = view["title"].apply(lambda x: re.sub("\d\d\d-", "", x).lower())
view = view.drop_duplicates("title")
view = view[view["CID"].duplicated(keep=False)]
view = view.sort_values("CID")
view = view.groupby("CID").filter(lambda x: len(x) == 4).head(n)
view.index = range(len(view))

In [24]:
Block = CharNgramBlock()
block_dict, block_ids = Block.block(view["title"], n_blocks=20)
pd.Series(block_ids).value_counts()

9     111
18     89
4      69
5      68
1      61
12     58
19     53
3      51
17     50
0      49
7      47
2      43
13     41
15     40
14     36
8      36
6      27
16     26
10     25
11     20
Name: count, dtype: int64

In [46]:
char_ngram_sim = CharNgramSim()
simmat_char_ngram = Simmat(sim_object=char_ngram_sim)
for i, t in enumerate(view["title"]):
    if i % 100 == 0:
        print(i)
    block = []
    for j in block_dict[block_ids[i]]:
        if j < i:
            block.append(j)
        else:
            break
    block = [j for j in block_dict[block_ids[i]] if j < i]  # todo improve?
    simmat_char_ngram.add_node(t, block)


edit_sim = EditSim()
simmat_edit = Simmat(sim_object=edit_sim)
for i, t in enumerate(view["title"]):
    if i % 100 == 0:
        print(i)
    block = []
    for j in block_dict[block_ids[i]]:
        if j < i:
            block.append(j)
        else:
            break
    block = [j for j in block_dict[block_ids[i]] if j < i]  # todo improve?
    simmat_edit.add_node(t, block)


tfidf_sim = TfidfSim(nodes=[i for i in view["title"].values])
simmat_tfidf = Simmat(sim_object=tfidf_sim)
for i, t in enumerate(view["title"]):
    if i % 100 == 0:
        print(i)
    block = []
    for j in block_dict[block_ids[i]]:
        if j < i:
            block.append(j)
        else:
            break
    block = [j for j in block_dict[block_ids[i]] if j < i]  # todo improve?
    simmat_tfidf.add_node(t, block)


partial_sim = EditPartialSim()
simmat_partial = Simmat(sim_object=partial_sim)
for i, t in enumerate(view["title"]):
    if i % 100 == 0:
        print(i)
    block = []
    for j in block_dict[block_ids[i]]:
        if j < i:
            block.append(j)
        else:
            break
    block = [j for j in block_dict[block_ids[i]] if j < i]  # todo improve?
    simmat_partial.add_node(t, block)


partial_set_sim = EditSetSim()
simmat_partial_set = Simmat(sim_object=partial_set_sim)
for i, t in enumerate(view["title"]):
    if i % 100 == 0:
        print(i)
    block = []
    for j in block_dict[block_ids[i]]:
        if j < i:
            block.append(j)
        else:
            break
    block = [j for j in block_dict[block_ids[i]] if j < i]  # todo improve?
    simmat_partial_set.add_node(t, block)

0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900


In [39]:
use_clique = True
simnet = SimNet(nodes=simmat_char_ngram.nodes, sim_mat=(0.7 * simmat_char_ngram.mat + 0.3 * simmat_edit.mat), edge_thresh=0.3, use_cliques=True)
print(simnet.g)

Graph with 1000 nodes and 7788 edges


In [15]:
"""
from pyvis.network import Network


net = Network(notebook=True, cdn_resources='remote')
for node in simnet.g.nodes:
    net.add_node(n_id=node, label=str(node), value=1)
for edge in simnet.g.edges:
    net.add_edge(edge[0], edge[1])
net.show('graph.html')
"""

"\nfrom pyvis.network import Network\n\n\nnet = Network(notebook=True, cdn_resources='remote')\nfor node in simnet.g.nodes:\n    net.add_node(n_id=node, label=str(node), value=1)\nfor edge in simnet.g.edges:\n    net.add_edge(edge[0], edge[1])\nnet.show('graph.html')\n"

In [47]:
# build dataset
if use_clique:
    clique = []
component = []
cn = []
pa = []
path = []
pairs = []
label = []
textsim_char_ngram = []
textsim_edit = []
textsim_tfidf = []
textsim_partial = []
textsim_set = []

label_dict = pd.Series(view["CID"].values, index=view["title"].values)
nodes = view["title"].values
for i in range(len(nodes)):
    if i % 100 == 0:
        print(i)
    for j in block_dict[block_ids[i]]:
        if i != j:
            node_i = nodes[i]
            node_j = nodes[j]
            if use_clique:
                clique.append(simnet.sim_clique_jaccard(node_i, node_j))
            component.append(simnet.sim_component(node_i, node_j))
            cn.append(simnet.sim_cn_jaccard(node_i, node_j, False))
            pa.append(simnet.sim_preferential_attachment(node_i, node_j))
            path.append(simnet.shortest_path(node_i, node_j))
            pairs.append([node_i, node_j])
            textsim_edit.append(simmat_edit.mat[i, j])
            textsim_char_ngram.append(simmat_char_ngram.mat[i, j])
            label.append(int(label_dict[node_i] == label_dict[node_j]))
            textsim_tfidf.append(simmat_tfidf.mat[i, j])
            textsim_partial.append(simmat_partial.mat[i, j])
            textsim_set.append(simmat_partial_set.mat[i, j])
if use_clique:
    df = pd.DataFrame({"pairs": pairs, "label": label, "component": component, "cn": cn, "pa": pa, "path": path, "textsim_edit": textsim_edit, "textsim_char_ngram": textsim_char_ngram, "clique": clique, "textsim_tfidf": textsim_tfidf, "textsim_partial": textsim_partial, "textsim_set": textsim_set})
else:
    df = pd.DataFrame({"pairs": pairs, "label": label, "component": component, "cn": cn, "pa": pa, "path": path, "textsim_edit": textsim_edit, "textsim_char_ngram": textsim_char_ngram, "textsim_tfidf": textsim_tfidf, "textsim_partial": textsim_partial, "textsim_set": textsim_set})
# impute & scale
df["path"] = 1 - df["path"].fillna(df["path"].max() + 1) / (df["path"].max() + 1)
df["pa"] = df["pa"] / df["pa"].max()
df

0
100
200
300
400
500
600
700
800
900


,pairs,label,component,cn,pa,path,textsim_edit,textsim_char_ngram,clique,textsim_tfidf,textsim_partial,textsim_set
0,"[die schildkroten, die schildkröten - kribbel-...",1,0.383929,0.620690,0.136973,0.714286,0.40,0.50,0.333333,0.09,94.0,64.0
1,"[die schildkroten, die schildkröten]",1,0.383929,0.916667,0.131266,0.714286,0.06,0.79,1.000000,0.16,94.0,97.0
2,"[die schildkroten, joe tabu - diee schildkröten]",1,0.383929,0.560000,0.091315,0.714286,0.32,0.50,0.285714,0.00,88.0,73.0
3,"[die schildkroten, 03 03 für 'ne moment]",0,0.383929,0.176471,0.097022,0.571429,0.72,0.00,0.000000,0.00,38.0,32.0
4,"[die schildkroten, wolfgang niedecken - 03 für...",0,0.383929,0.181818,0.091315,0.571429,0.78,0.00,0.000000,0.00,38.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58379,"[prélude no. 6 h-moll, op. 28: lento assai, so...",0,0.607143,0.318182,0.051613,0.571429,0.84,0.02,0.000000,0.00,24.0,24.0
58380,"[prélude no. 6 h-moll, op. 28: lento assai, so...",0,0.607143,0.280000,0.063524,0.571429,0.87,0.02,0.000000,0.00,24.0,24.0
58381,"[prélude no. 6 h-moll, op. 28: lento assai, fr...",1,0.607143,0.631579,0.059553,0.714286,0.18,0.82,0.437500,0.83,100.0,100.0
58382,"[prélude no. 6 h-moll, op. 28: lento assai, pr...",1,0.607143,0.705882,0.051613,0.714286,0.02,0.95,0.636364,0.77,98.0,99.0


In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score


# train test split, first 20% of nodes are treated as new at inference
nodes_test = nodes[:int(len(nodes) * 0.2)]
df["contains_node_test"] = df["pairs"].apply(lambda pair: (pair[0] in nodes_test) or (pair[1] in nodes_test))
df_test = df[df["contains_node_test"]].drop(["contains_node_test", "pairs"], axis=1)
df_train = df[~df["contains_node_test"]].drop(["contains_node_test", "pairs"], axis=1)
x_train = df_train.drop("label", axis=1)
y_train = df_train["label"]
x_test = df_test.drop("label", axis=1)
y_test = df_test["label"]

# train / predict
cls = RandomForestClassifier().fit(x_train, y_train)
preds = cls.predict(x_test)
print(recall_score(y_pred=preds, y_true=y_test))
print(precision_score(y_pred=preds, y_true=y_test))

0.9776951672862454
0.8855218855218855


In [49]:
pd.DataFrame({"feature": x_train.columns, "importance": cls.feature_importances_}).sort_values("importance")

,feature,importance
3,path,0.000252
0,component,0.004551
1,cn,0.012193
2,pa,0.017236
6,clique,0.018906
4,textsim_edit,0.033962
7,textsim_tfidf,0.164458
8,textsim_partial,0.191459
5,textsim_char_ngram,0.221813
9,textsim_set,0.335169
